In [ ]:
library(Seurat)
library(GSVA)

library(ggplot2)
library(ggpubr)
library(ggtext)
library(ggrepel)
library(ComplexHeatmap)
library(cowplot)

library(reshape2)

library(survival)
library(survminer)

In [ ]:
#路径
seu <- readRDS("")                                              #单细胞seurat rds对象位置
meta <- read.csv("", row.names = 1)                             #scRank预测结果
figurePath <- ""                                                #数据保存位置
if (!dir.exists(figurePath)) {
    dir.create(figurePath, recursive = T)
}
dataPath <- ""                                                  #bulk数据位置

pvalue_threshold=0.01                                           #findmarker p值阈值

log2FoldChange_threshold=1                     #log2FoldChange_threshold阈值，高于log2FoldChange_threshold被定义为NR相关基因

In [ ]:
# 函数定义
# 火山图函数
plot_Volcano_2 <- function(result, logFC = 0.5, p_val = 0.05, label_geneset = NULL) {
    result$change <- "NONE"
    result$change[which(result$log2FoldChange >= logFC & result$p_val <= p_val)] <- "UP"
    result$change[which(result$log2FoldChange <= (-logFC) & result$p_val <= p_val)] <- "DOWN"
    xlim <- max(abs(result$log2FoldChange))
    if (is.null(label_geneset)) {
        p <- ggplot(result, aes(x = log2FoldChange, y = -log10(p_val))) +
            geom_point(data = result, aes(x = log2FoldChange, y = -log10(p_val), color = change)) +
            theme_bw() +
            geom_vline(xintercept = c(-logFC, logFC), lty = 2) +
            geom_hline(yintercept = c(-log10(p_val)), lty = 2) +
            scale_x_continuous(limits = c(-xlim, xlim)) +
            coord_fixed(ratio = (2 * xlim) / (max(-log10(result$p_val), na.rm = T))) +
            theme(
                panel.grid = element_blank(), legend.title = element_blank(),
                panel.grid.minor = element_blank(),
                axis.text = element_text(color = "black")
            ) +
            xlab("log2FoldChange") +
            ylab("-log10P-value") +
            scale_color_manual(values = c("NONE" = "grey", "UP" = "red", "DOWN" = "blue")) +
            guides(fill = FALSE)
    } else {
        p <- ggplot(result, aes(x = log2FoldChange, y = -log10(p_val))) +
            geom_point(data = result, aes(x = log2FoldChange, y = -log10(p_val), color = change)) +
            geom_label_repel(
                data = result[which(result$gene %in% label_geneset), ],
                aes(x = log2FoldChange, y = -log10(p_val), label = gene, fill = change), color = "white", fontface = "italic"
            ) +
            theme_bw() +
            geom_vline(xintercept = c(-logFC, logFC), lty = 2) +
            geom_hline(yintercept = c(-log10(p_val)), lty = 2) +
            scale_x_continuous(limits = c(-xlim, xlim)) +
            coord_fixed(ratio = (2 * xlim) / (max(-log10(result$p_val), na.rm = T))) +
            theme(
                panel.grid = element_blank(), legend.title = element_blank(),
                panel.grid.minor = element_blank(),
                axis.text = element_text(color = "black")
            ) +
            xlab("log2FoldChange") +
            ylab("-log10P-value") +
            scale_color_manual(values = c("UP" = "red", "DOWN" = "blue", "NONE" = "grey")) +
            scale_fill_manual(values = c("UP" = "red", "DOWN" = "blue", "NONE" = "grey")) +
            guides(fill = FALSE)
    }
    return(p)
}

## GSEA function
GSEAfunc <- function(meta, score) {
    data_combined <- cbind(meta, score = as.numeric(score))
    p <- ggboxplot(data_combined,
        x = "Response", y = "score",
        color = "Response", palette = c("#00AFBB", "#E7B800"),
        add = "jitter"
    )
    my_comparisons <- list(c("R", "NR"))
    p <- p + stat_compare_means(comparisons = my_comparisons, label = "p.label", size = 5, method = "wilcox.test")

    return(p)
}

In [ ]:
colnames(meta) = c("Reject", "Pred_score", "Predict")
table(meta$Predict)
seu <- AddMetaData(seu, meta)

In [ ]:
# DimPlot
p=DimPlot(seu, group.by = c("Predict"), cols = c("Background" = "gray91", "Rank-" = "#4cb1c4", "Rank+" = "#b5182b"))
png(paste0(figurePath, "UMAP-Rank.png"))
print(p)
dev.off()

In [ ]:
# 差异基因火山图
Idents(seu) <- seu$Predict
all_marker <- FindMarkers(seu, ident.1 = "Rank+", ident.2 = "Rank-")

marker <- all_marker[all_marker$p_val <= pvalue_threshold, ]

marker <- marker[order(marker$avg_log2FC, decreasing = TRUE), ]
marker$gene <- rownames(marker)

colnames(marker) <- c("p_val", "log2FoldChange", "pct.1", "pct.2", "padj", "gene")
saveRDS(marker, paste0(figurePath, "marker.rds"))


p <- plot_Volcano_2(marker, label_geneset = c(head(marker, 5)[, "gene"], tail(marker, 5)[, "gene"]), logFC = 0.5)
png(paste0(figurePath, "DEGs Volcano.png"), width = 600, height = 800)
print(p)
dev.off()

In [ ]:
#循环富集

marker <- readRDS(paste0(figurePath, "marker.rds"))
NRmarkers <- marker[marker$log2FoldChange >= log2FoldChange_threshold, "gene"]
Rmarkers <- marker[marker$log2FoldChange <= -log2FoldChange_threshold, "gene"]

NRmarkers <- list(NRmarkers)
Rmarkers <- list(Rmarkers)


datasets <- sapply(list.files(dataPath), function(x) {
    return(strsplit(x, "_")[[1]][1])
})
datasets <- unique(unname(datasets))

In [ ]:
for (data in datasets) {
    
    exp <- read.csv(paste0(dataPath, data, "_exp.csv"), row.names = 1)
    meta <- read.csv(paste0(dataPath, data, "_meta.csv"), row.names = 1)
    exp <- as.matrix(exp)
    meta[meta$Response%in%c("CRPR"),"Response"]="R"
    meta[meta$Response%in%c("SD","PD"),"Response"]="NR"

    NRscore <- gsva(exp, gset.idx.list = NRmarkers, method = "ssgsea")
    Rscore <- gsva(exp, gset.idx.list = Rmarkers, method = "ssgsea")

    #富集
    p1 <- GSEAfunc(meta, NRscore)
    png(paste0(figurePath, data, " NR genes enrich boxplot.png"), width = 450, height = 450)
    print(p1)
    dev.off()
    p1 <- GSEAfunc(meta, Rscore)
    png(paste0(figurePath, data, " R genes enrich boxplot.png"), width = 450, height = 450)
    print(p1)
    dev.off()





    ## 生存分析
    data_combined <- cbind(meta, NRscore = as.numeric(NRscore))
    data_combined <- cbind(data_combined, Rscore = as.numeric(Rscore))
    NRscore_cutoff <- surv_cutpoint(data = data_combined, time = "OS_time", event = "OS_status", variables = "NRscore")$cutpoint[1, 1]
    Rscore_cutoff <- surv_cutpoint(data = data_combined, time = "OS_time", event = "OS_status", variables = "Rscore")$cutpoint[1, 1]

    data_combined$NRscore_group <- ifelse(data_combined$NRscore > NRscore_cutoff, "High", "Low")
    data_combined$Rscore_group <- ifelse(data_combined$Rscore > Rscore_cutoff, "High", "Low")

    fit1 <- survfit(Surv(time = data_combined$OS_time, event = data_combined$OS_status) ~ NRscore_group, data = data_combined)
    fit2 <- survfit(Surv(time = data_combined$OS_time, event = data_combined$OS_status) ~ Rscore_group, data = data_combined)

    p <- ggsurvplot(fit1,
        data = data_combined,
        pval = TRUE, # 如果你想显示p值
        risk.table = TRUE
    ) # 如果你想显示置信区间
    png(paste0(figurePath, data, " NR genes enrichment score with OS.png"), width = 450, height = 450)
    print(p)
    dev.off()

    p <- ggsurvplot(fit2,
        data = data_combined,
        pval = TRUE, # 如果你想显示p值
        risk.table = TRUE
    ) # 如果你想显示置信区间
    png(paste0(figurePath, data, " R genes enrichment score with OS.png"), width = 450, height = 450)
    print(p)
    dev.off()
    
}

## scRank预测亚群分布

In [ ]:
freq_table <- table(seu$seurat_clusters, seu$Predict)

df <- as.data.frame(freq_table)
cluster_totals <- aggregate(Freq ~ Var1, data = df, sum)

df <- merge(df, cluster_totals, by = "Var1")

df$Proportion <- df$Freq.x / df$Freq.y
df=df[df$Var2=="Rank+",]
df=df[order(df$Proportion,decreasing = T),] #根据scRank+细胞比例排序
level=df$Var1

In [ ]:
df=as.data.frame(table(seu$seurat_clusters,seu$Predict))
cluster_totals <- aggregate(Freq ~ Var1, data = df, sum)
df <- merge(df, cluster_totals, by = "Var1")
df$Proportion <- df$Freq.x / df$Freq.y
colnames(df)=c("cluster","Rank_Label","number","total_number","Proportion")
df=df[df$Rank_Label!="Background",]
df$cluster=factor(df$cluster,levels=level)

In [ ]:
p=ggplot(data = df,aes(x=cluster,y=Proportion,fill=Rank_Label))+
  geom_bar(stat = "identity")+
    scale_fill_manual(values = c('Rank-'='#4cb1c4','Rank+'='#b5182b'),limits=c("Rank+","Rank-"))+theme_classic()+  
    theme(panel.background = element_blank(),axis.line = element_line())

png(paste0(figurePath, "scRank cluster distrubution.png"), width = 600, height = 800)
print(p)
dev.off()    

## 特定亚群分析（CD8+ T细胞为例）

In [ ]:
subcluster=c('0','1','3','4','7','8','11')

In [ ]:
seu_CD8=subset(seu,seurat_clusters%in%subcluster)
Idents(seu_CD8)=seu_CD8$Predict
marker=FindMarkers(seu_CD8,ident.1 = "Rank+", ident.2 = "Rank-")
marker <- marker[marker$p_val < pvalue_threshold, ]
# 根据logFC排序
marker <- marker[order(marker$avg_log2FC, decreasing = TRUE), ]
colnames(marker)=c('p_val','log2FoldChange','pct.1','pct.2','padj')
marker$gene=rownames(marker)

p=plot_Volcano_2(marker,label_geneset=c(head(rownames(marker),5),tail(rownames(marker),5)),logFC=0.5)
png(paste0(figurePath, "subcluster_DEGs Volcano.png"), width = 600, height = 800)
print(p)
dev.off()